- modify truncated validation again

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, cohen_kappa_score
import optuna.integration.lightgbm as lgb_opt
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from hyperopt import hp, tpe, Trials, fmin, space_eval
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder_cla(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        #return -mod_qwk(y, X_p, weights=weights)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(0.4,0.5), np.random.uniform(0.5,0.6), np.random.uniform(0.6,0.7)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead') #Powell
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']
    
class OptimizedRounder_reg(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        #return -mod_qwk(y, X_p, weights=weights)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(1.0,1.1), np.random.uniform(1.7,1.8), np.random.uniform(2.1,2.2)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead') #Powell
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

In [4]:
def eval_qwk_lgb_regr(y_pred, train_t):
    dist = Counter(train_t['accuracy_group'])
    for k in dist:
        dist[k] /= len(train_t)
    
    acum = 0
    bound = {}
    for i in range(3):
        acum += dist[i]
        bound[i] = np.percentile(y_pred, acum * 100)

    def classify(x):
        if x <= bound[0]:
            return 0
        elif x <= bound[1]:
            return 1
        elif x <= bound[2]:
            return 2
        else:
            return 3

    y_pred = np.array(list(map(classify, y_pred)))
    
    return y_pred

# install

In [5]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 16s, sys: 7.94 s, total: 1min 24s
Wall time: 1min 24s


# Preprocess and Feature engineering

In [6]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = sorted(list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique()))))
    
    train['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), train['type'], train['world']))
    test['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), test['type'], test['world']))
    list_of_type_world = sorted(list(set(train['type_world'].unique()).union(set(test['type_world'].unique()))))
    
    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
        
    train["true"] = train["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)
    test["true"] = test["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)

    train["false"] = train["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    test["false"] = test["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    
    train["game_complete"] = train["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    test["game_complete"] = test["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    
    #train["level"] = train["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    #test["level"] = test["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    
    #train["round"] = train["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
    #test["round"] = test["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
               
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world = encode_title(train, test)

CPU times: user 1min 49s, sys: 8.78 s, total: 1min 58s
Wall time: 1min 57s


In [7]:
def make_ratio(features, dic):
    total = sum(dic.values())
    if total != 0:
        for key in dic.keys():
            features[str(key)] = features[str(key)] / total
    else:
        pass
    return features

def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    event_code_count = {str(ev): 0 for ev in list_of_event_code}
    title_count = {actv: 0 for actv in list_of_user_activities}
    type_world_count = {str(ev): 0 for ev in list_of_type_world}
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    last_game_time_title = {'lgt_' + title: 0 for title in assess_titles}
    ac_game_time_title = {'agt_' + title: 0 for title in assess_titles}
    ac_true_attempts_title = {'ata_' + title: 0 for title in assess_titles}
    ac_false_attempts_title = {'afa_' + title: 0 for title in assess_titles}
    
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = user_sample.iloc[0]['timestamp']
    miss = 0
    crys_game_true = 0; crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    durations = []
    prev_assess_title = -999
    assess_count = 1
    last_accuracy = -999
    prev_assess_start = -999; prev_assess_end = -999
    real_prev_assess_start = -999; real_prev_assess_end = -999
    real_assess_start = -999; real_assess_end = -999
    complete_games = 0
    no_result_count = 0
    crys_game_level = np.array([]); tree_game_level = np.array([]); magma_game_level = np.array([])
    crys_game_round = np.array([]); tree_game_round = np.array([]); magma_game_round = np.array([])
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        if session_type != 'Assessment':
            if session_type == "Game":
                true = session['true'].sum()
                false = session['false'].sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #crys_game_level = np.concatenate([crys_game_level, session["level"]], axis=0)
                    #crys_game_round = np.concatenate([crys_game_round, session["round"]], axis=0)
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #tree_game_level = np.concatenate([tree_game_level, session["level"]], axis=0)
                    #tree_game_round = np.concatenate([tree_game_round, session["round"]], axis=0)
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #magma_game_level = np.concatenate([magma_game_level, session["level"]], axis=0)
                    #magma_game_round = np.concatenate([magma_game_round, session["round"]], axis=0)
                else:
                    pass
                
        if (session_type == 'Assessment') & (test_set or len(session)>1): 
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            assess_start = session.iloc[0,2]
            assess_end = session.iloc[-1,2]
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features = make_ratio(features, user_activities_count)
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features = make_ratio(features, title_eventcode_count)
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features = make_ratio(features, user_world_count)
            features.update(event_code_count.copy())
            features = make_ratio(features, event_code_count)
            features.update(title_count.copy())
            features = make_ratio(features, title_count)
            features.update(type_world_count.copy())
            features = make_ratio(features, type_world_count)
            features.update(last_accuracy_title.copy())
            features.update(last_game_time_title.copy())
            features.update(ac_game_time_title.copy())
            features.update(ac_true_attempts_title.copy())
            features.update(ac_false_attempts_title.copy())
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            ac_true_attempts_title['ata_' + session_title_text] += true_attempts
            ac_false_attempts_title['afa_' + session_title_text] += false_attempts
            last_game_time_title['lgt_' + session_title_text] = session['game_time'].iloc[-1]
            ac_game_time_title['agt_' + session_title_text] += session['game_time'].iloc[-1]
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            features["no_complete_game"] = complete_games
            features["no_result_count"] = no_result_count 
            
            if true_attempts + false_attempts == 0:
                no_result_count += 1
            else:
                real_assess_start = session.iloc[0,2]
                real_assess_end = session.iloc[-1,2]
             
            #features["crys_game_true"] = crys_game_true
            #features["crys_game_false"] = crys_game_false
            #features['crys_game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
            #features["crys_game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
            #features["cryslast_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            #features["tree_game_true"] = tree_game_true
            #features["tree_game_false"] = tree_game_false
            #features['tree_game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
            #features["tree_game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
            #features["tree_last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            #features["magma_game_true"] = magma_game_true
            #features["magma_game_false"] = magma_game_false
            #features['magma_game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
            #features["magma_game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
            #features["magma_last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            
            if session_world == activities_world["CRYSTALCAVES"]:
                features["game_true"] = crys_game_true
                features["game_false"] = crys_game_false
                features['game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            elif session_world == activities_world["TREETOPCITY"]:
                features["game_true"] = tree_game_true
                features["game_false"] = tree_game_false
                features['game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            elif session_world == activities_world["MAGMAPEAK"]:
                features["game_true"] = magma_game_true
                features["game_false"] = magma_game_false
                features['game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            features["prev_assess_title"] = prev_assess_title
            prev_assess_title = session_title
            features["first_assessment"] = 1 if assess_count == 1 else 0
            #features["assess_count"] = assess_count
            assess_count += 1
            features["time_from_start"] = (assess_start - time_first_activity).seconds

            if prev_assess_end == -999:
                features["time_bet_assess"] = -999
            else:
                features["time_bet_assess"] = (assess_start - prev_assess_end).seconds
            prev_assess_start = assess_start
            prev_assess_end = assess_end
            if real_prev_assess_end == -999:
                features["time_bet_real_assess"] = -999
            else:
                features["time_bet_real_assess"] = (real_assess_start - real_prev_assess_end).seconds
            real_prev_assess_start = real_assess_start
            real_prev_assess_end = real_assess_end
            
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
                features['duration_std'] = 0
                features['duration_max'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                features['duration_max'] = np.max(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds)
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            features['last_assess_acc'] = last_accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            last_accuracy = accuracy
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
            
        complete_games += np.sum(session["game_complete"])
        miss += np.sum(session["misses"])
        user_world_count["world_"+str(session_world)] += session.shape[0]
        
        n_of_type_world = Counter(session['type_world']) 
        for key in n_of_type_world.keys():
            type_world_count[str(key)] += n_of_type_world[key]
            
        n_of_title = Counter(session['title']) 
        for key in n_of_title.keys():
            title_count[activities_labels[key]] += n_of_title[key]
            
        n_of_eventcode = Counter(session['event_code']) 
        for key in n_of_eventcode.keys():
            event_code_count[str(key)] += n_of_eventcode[key]
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1], all_assessments[:-1] # test previous data to incorporate into training
    return all_assessments

In [8]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val = []

    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
        compiled_train += get_data(user_sample)
    del train
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
        test_data, val_data = get_data(user_sample, test_set=True)
        compiled_test.append(test_data)
        compiled_val += val_data
    del test
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)

    categoricals = ['session_title']
    return reduce_train, reduce_test, reduce_val, categoricals
new_train, new_test, new_val, categoricals = get_train_and_test(train, test)

In [9]:
tmp = new_train[new_train.Game==0].copy()
tmp = tmp[tmp.Activity == 0].copy()
tmp = tmp[tmp.Clip == 0].copy()
tmp = tmp[tmp.Assessment ==0].copy()
remove_train_index = tmp.index
new_train = new_train[~new_train.index.isin(remove_train_index)].copy()

In [10]:
print(new_train.shape)
print(new_test.shape)
print(new_val.shape)

(17577, 563)
(1000, 563)
(2347, 563)


In [11]:
# data augmentation
tmp = new_val[new_val.Game==0].copy()
tmp = tmp[tmp.Activity == 0].copy()
tmp = tmp[tmp.Clip == 0].copy()
tmp = tmp[tmp.Assessment ==0].copy()
remove_val_index = tmp.index
add_val = new_val[~new_val.index.isin(remove_val_index)].copy()

#tmp = add_val.installation_id.value_counts().reset_index(drop=False) # include some part of new_test installation_id
#val_id = list(tmp[tmp.installation_id >= 20]["index"])
#add_val = add_val[add_val.installation_id.isin(val_id)]

mod_train = pd.concat([new_train, add_val], ignore_index=True)
mod_train.shape

(19906, 563)

# Feature selection

In [12]:
def exclude(reduce_train, reduce_test, features):
    to_exclude = [] 
    ajusted_test = reduce_test.copy()
    for feature in features:
        if feature not in ['accuracy_group', 'installation_id', 'session_title', 'hightest_level']:
            data = reduce_train[feature]
            train_mean = data.mean()
            data = ajusted_test[feature] 
            test_mean = data.mean()
            try:
                ajust_factor = train_mean / test_mean
                if ajust_factor > 10 or ajust_factor < 0.1:# or error > 0.01:
                    to_exclude.append(feature)
                    print(feature)
                else:
                    ajusted_test[feature] *= ajust_factor
            except:
                to_exclude.append(feature)
                print(feature)
    return to_exclude, ajusted_test
features = [i for i in new_train.columns if i not in ["game_session"]]
to_exclude, ajusted_test = exclude(new_train, new_test, features)

Air Show_4080
Bottle Filler (Activity)_2010
Bubble Bath_4080
Bubble Bath_4090
Bug Measurer (Activity)_4080
Cart Balancer (Assessment)_4080
Chest Sorter (Assessment)_4080
Crystals Rule_2010
Dino Dive_4080
Dino Drink_4080
Egg Dropper (Activity)_4080
Fireworks (Activity)_4080
Happy Camel_4080
Leaf Leader_4080
Mushroom Sorter (Assessment)_4080
Pan Balance_2010
Pan Balance_4080
Sandcastle Builder (Activity)_2010
Scrub-A-Dub_4080
Watering Hole (Activity)_2010
acc_Cart Balancer (Assessment)


# classification

In [13]:
def feature_selection(train):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    features_list = [i for i in X_train.columns if i != "installation_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
            
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
        test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid.iloc[test_index] = test_predict.reshape(X_test2.shape[0], 1)
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    return feature_importance_df
#df_for_classification = feature_selection(new_train)

In [14]:
def accuracy_class(train, test, fea, select_flg):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude ]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    print(X_test.shape[1])
    
    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
        if select_flg == True:
            X_train2 = X_train2[fea] 
            X_test2 = X_test2[fea]
            
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
        test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        models.append(clf)
        valid.iloc[test_index] = test_predict.reshape(X_test2.shape[0], 1)
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    print('Accuracy score = \t {}'.format(accuracy_score(y_train, np.round(valid))))
    print('Precision score = \t {}'.format(precision_score(y_train, np.round(valid))))
    print('Recall score =   \t {}'.format(recall_score(y_train, np.round(valid))))
    print('F1 score =      \t {}'.format(f1_score(y_train, np.round(valid))))
    print(confusion_matrix(y_train, np.round(valid)))
    pred_value = np.zeros([X_test.shape[0]])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return pred_value, valid

#tmp = df_for_classification.sort_values("Cv", ascending = True).reset_index(drop=True).copy()
#feat = tmp[tmp.index <= 120]["Feature"]
#feat = []
#pred_value, valid = accuracy_class(new_train, new_test, feat, False)
#pred_value, valid = accuracy_class(mod_train, new_test, feat, False)

In [15]:
#best_score = 0
#for i in range(10):
#    optR = OptimizedRounder_cla()
#    #optR.fit(np.array(valid).reshape(-1,), new_train.accuracy_group, random_flg=True)
#    optR.fit(np.array(valid).reshape(-1,), mod_train.accuracy_group, random_flg=True)
#    coefficients = optR.coefficients()
#    final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
    #score = qwk(new_train.accuracy_group, final_valid_pred)
    #score = qwk(mod_train.accuracy_group, final_valid_pred)
#    print(i, np.sort(coefficients), score)
#    if score > best_score:
#        best_score = score
#        best_coefficients = coefficients
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(best_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.31635244, 0.54903181, 0.74679975])) + [np.inf], labels = [0, 1, 2, 3])

#sample_submission["accuracy_group"] = final_test_pred.astype(int)
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission["accuracy_group"].value_counts(normalize = True)

# truncated version

In [16]:
def feature_selection_mod(train):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    features_list = [i for i in X_train.columns if i != "installation_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    random_try = 10
    for try_time in range(random_try):
        valid = np.array([])
        real = np.array([])
        for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
            print("Fold "+str(i+1))
            X_train2 = X_train.iloc[train_index,:]
            y_train2 = y_train.iloc[train_index]
            X_train2 = X_train2.drop(['installation_id'],axis=1)
    
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            X_test2, idx_val = get_random_assessment(X_test2)
            #tmp_target = X_test2.loc[idx_val]["past_target"]
            X_test2.drop(['installation_id'], inplace=True, axis=1) # 'past_target'
            y_test2 = y_test2.loc[idx_val]
            print("After truncation:", (X_test2.shape, y_test2.shape))
            
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
            valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], )
            valid = np.concatenate([valid, valid_predict])
            real = np.concatenate([real, y_test2])
            feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
                
        print("logloss = \t {}".format(log_loss(real, valid)))
        print("ROC = \t {}".format(roc_auc_score(real, valid)))
        feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    return feature_importance_df
#df_for_classification = feature_selection_mod(new_train)

In [17]:
def get_random_assessment(reduce_train):
    used_idx = []
    for iid in tqdm(set(reduce_train['installation_id']), miniters=200):
        list_ = list(reduce_train[reduce_train['installation_id'] == iid].index)
        cur = random.choices(list_, k=1)[0]
        used_idx.append(cur)
    reduce_train_t = reduce_train.loc[used_idx]
    return reduce_train_t, used_idx

def accuracy_class_mod(train, test, fea, select_flg):
    #X_train = train.drop(["accuracy_group"],axis=1) 
    y_train = train.accuracy_group.copy()
    X_train = train.rename(columns={"accuracy_group": "past_target"})
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    
    n_folds = 10
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.5698056418890787,'bagging_freq': 4,
 'colsample_bytree': 0.37564408454469,'learning_rate': 0.015433389422506185,'max_depth': 8,
 'min_data_in_leaf': 51,'min_sum_hessian_in_leaf': 10,'num_leaves': 48}
    random_try = 20
    mean_qwk_score = 0
    # model learning ---------------------------
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id', 'past_target'],axis=1)
    
        for try_time in range(random_try):
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            # random truncation ------------------------------------------
            X_test2, idx_val = get_random_assessment(X_test2)
            tmp_target = X_test2.loc[idx_val]["past_target"]
            X_test2.drop(['installation_id', 'past_target'], inplace=True, axis=1)
            y_test2 = y_test2.loc[idx_val]
            print("After truncation:", (X_test2.shape, y_test2.shape))
            # ------------------------------------------------------------
            
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
            valid = np.array(clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], ))
            real = np.array(y_test2)
            target = np.array(tmp_target)
            
            models.append(clf)
            print("logloss = \t {}".format(log_loss(real, valid)))
            print("ROC = \t {}".format(roc_auc_score(real, valid)))
        
            # threshold optimization --------------
            best_score = 0
            for j in range(20):
                optR = OptimizedRounder_cla()
                optR.fit(np.array(valid).reshape(-1,), target, random_flg=True)
                coefficients = optR.coefficients()
                final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
                score = qwk(target, final_valid_pred)
                print(j, np.sort(coefficients), score)
                if score > best_score:
                    best_score = score
                    best_coefficients = coefficients
            mean_qwk_score += best_score / (random_try * n_folds)
            if try_time == 0 and i == 0:
                final_coefficients = np.sort(best_coefficients) / (random_try * n_folds)
            else:
                final_coefficients += np.sort(best_coefficients) / (random_try * n_folds)
            
    print("MEAN QWK = \t {}".format(mean_qwk_score))
    # test prediction  ------------------------
    pred_value = np.zeros([X_test.shape[0]])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return pred_value, valid, final_coefficients

#tmp = df_for_classification.sort_values("Cv", ascending = True).reset_index(drop=True).copy()
#feat = tmp[tmp.index <= 120]["Feature"]
feat = []
pred_value, valid, final_coefficients = accuracy_class_mod(new_train, new_test, feat, False)
final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(final_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.32229148, 0.51887455, 0.77529457])) + [np.inf], labels = [0, 1, 2, 3])

sample_submission["accuracy_group"] = final_test_pred.astype(int)
sample_submission.to_csv('submission.csv', index=False)
sample_submission["accuracy_group"].value_counts(normalize = True)

Fold 1



After truncation: ((355, 538), (355,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	training's auc: 0.833112	valid_1's auc: 0.765607
logloss = 	 0.6453514371629447
ROC = 	 0.7656067778006448
0 [0.40450731 0.62742514 0.63294673] 0.40534463
1 [0.47222079 0.62741556 0.63305401] 0.40534463
2 [0.47727619 0.51104299 0.6330965 ] 0.22385254
3 [0.47365483 0.50049023 0.68904242] 0.0
4 [0.4929088  0.54648864 0.68871845] 0.0
5 [0.42035229 0.51519997 0.68133116] 0.0
6 [0.41844357 0.51342743 0.63292323] 0.22385254
7 [0.42286001 0.50104223 0.63302936] 0.22385254
8 [0.41692664 0.54509447 0.69717475] 0.0
9 [0.4274166  0.62583913 0.64914806] 0.27735133
10 [0.4612548  0.55817096 0.63299162] 0.22385254
11 [0.46435986 0.54415065 0.63304672] 0.22385254
12 [0.44769926 0.62596981 0.6319629 ] 0.40988818
13 [0.41644556 0.6256959  0.67893321] 0.28116954
14 [0.48563131 0.50016136 0.63304809] 0.22385254
15 [0.45439849 0.51509477 0.63292542] 0.22385254
16 [0.


After truncation: ((355, 538), (355,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[155]	training's auc: 0.888993	valid_1's auc: 0.829017
logloss = 	 0.5105659161795707
ROC = 	 0.8290166812993854
0 [0.42644788 0.63139788 0.70697602] 0.58399699
1 [0.39336049 0.56338931 0.69564872] 0.59911121
2 [0.42498871 0.56483145 0.70227403] 0.5820734
3 [0.3823678  0.56277296 0.6894326 ] 0.59465237
4 [0.50317733 0.62453407 0.70737274] 0.57152589
5 [0.39242657 0.56862816 0.70287711] 0.59400222
6 [0.32405101 0.56265106 0.69494681] 0.57936499
7 [0.42675208 0.63610164 0.69550297] 0.58699197
8 [0.36006037 0.63460516 0.69028446] 0.591942
9 [0.39187018 0.56369282 0.69602156] 0.59911121
10 [0.39253024 0.56221026 0.70243155] 0.59662397
11 [0.39413068 0.56099447 0.73365106] 0.59151212
12 [0.4062413  0.63491078 0.64713357] 0.57285568
13 [0.50411379 0.52430563 0.68970568] 0.56503765
14 [0.39127876 0.62519671 0.69174312] 0.59502365
15 [0.38287201 0.52326885 0.


After truncation: ((355, 538), (355,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	training's auc: 0.875535	valid_1's auc: 0.806837
logloss = 	 0.5554445917138018
ROC = 	 0.8068370119970836
0 [0.44590056 0.61335842 0.70133265] 0.54170698
1 [0.44459235 0.60679559 0.67853121] 0.53675511
2 [0.43885894 0.5969302  0.68038948] 0.53789427
3 [0.41953247 0.58562688 0.70479848] 0.5301085
4 [0.43901469 0.60300554 0.67850101] 0.53808236
5 [0.44545823 0.54191531 0.67952819] 0.51621327
6 [0.43933698 0.59624229 0.68134006] 0.5382289
7 [0.48793234 0.53347247 0.67815808] 0.51089017
8 [0.4199238  0.60165923 0.67842709] 0.53151669
9 [0.44559655 0.60183423 0.68167217] 0.53777401
10 [0.43755155 0.58877188 0.67971548] 0.53807118
11 [0.43706769 0.53310748 0.68130959] 0.51753621
12 [0.45464368 0.61253044 0.70513169] 0.53902306
13 [0.41980449 0.61194988 0.7050529 ] 0.53509036
14 [0.43578091 0.55229492 0.65423525] 0.51624837
15 [0.4224642  0.65275867 0.


After truncation: ((355, 538), (355,))
Training until validation scores don't improve for 100 rounds
[500]	training's auc: 0.93197	valid_1's auc: 0.839142
Early stopping, best iteration is:
[499]	training's auc: 0.931886	valid_1's auc: 0.839142
logloss = 	 0.4722929577525337
ROC = 	 0.8391418211120065
0 [0.43382234 0.51843878 0.72746596] 0.58028614
1 [0.44664401 0.54193214 0.61882628] 0.55004875
2 [0.45177813 0.54212256 0.62471998] 0.55004875
3 [0.43328779 0.50206412 0.75155008] 0.57805178
4 [0.49146221 0.54266136 0.61651378] 0.54088739
5 [0.40024389 0.488638   0.74390949] 0.57632176
6 [0.37301496 0.54795895 0.73148069] 0.58009666
7 [0.38914356 0.51294307 0.73169625] 0.57715055
8 [0.44228225 0.54136945 0.64305449] 0.55028041
9 [0.46501053 0.53770062 0.65791014] 0.55100549
10 [0.45259737 0.54230868 0.74394199] 0.57792354
11 [0.44867829 0.48806291 0.73258938] 0.57926031
12 [0.44747878 0.54572439 0.75102919] 0.57872679
13 [0.47397447 0.54160411 0.74852341] 0.57030374
14 [0.44896207 0.595

...


After truncation: ((358, 538), (358,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[45]	training's auc: 0.863642	valid_1's auc: 0.810717
logloss = 	 0.61347258827976
ROC = 	 0.8107166503767695
0 [0.49808997 0.59521152 0.6638296 ] 0.54245008
1 [0.54852925 0.62249884 0.7081217 ] 0.53477759
2 [0.49906777 0.59665761 0.66675328] 0.54521554
3 [0.5298877  0.59575938 0.67798947] 0.54202449
4 [0.52198536 0.583392   0.66665459] 0.54766414
5 [0.50284015 0.58101232 0.66725301] 0.54472273
6 [0.54972103 0.58143303 0.66693771] 0.55517115
7 [0.42230214 0.58371311 0.65669528] 0.46903335
8 [0.42934817 0.58396353 0.66719947] 0.48888978
9 [0.44849186 0.59580142 0.66663013] 0.49925048
10 [0.54953725 0.59552663 0.6789657 ] 0.54463646
11 [0.49880749 0.58143468 0.66663831] 0.54826498
12 [0.49970386 0.59678917 0.67855706] 0.5367649
13 [0.53419549 0.62040106 0.66812019] 0.54681982
14 [0.50966311 0.58152683 0.66682613] 0.54484875
15 [0.49775689 0.58338887 0.6


After truncation: ((358, 538), (358,))
Training until validation scores don't improve for 100 rounds
[500]	training's auc: 0.931157	valid_1's auc: 0.843601
Early stopping, best iteration is:
[454]	training's auc: 0.927082	valid_1's auc: 0.845617
logloss = 	 0.4930274978716565
ROC = 	 0.8456171735241502
0 [0.3081517  0.63260481 0.77159316] 0.61107938
1 [0.43764322 0.53264063 0.74325761] 0.60219447
2 [0.43699094 0.61960855 0.75801487] 0.61206717
3 [0.43334279 0.52021498 0.77100535] 0.60929046
4 [0.38248199 0.55686192 0.75736743] 0.61379596
5 [0.32891336 0.61931194 0.75222395] 0.61163286
6 [0.45874636 0.5346485  0.63507514] 0.56291415
7 [0.43598228 0.52365139 0.77116855] 0.61155071
8 [0.43928965 0.54521967 0.75796838] 0.60083574
9 [0.47246022 0.55768864 0.77120918] 0.61119945
10 [0.42358594 0.5484181  0.67721042] 0.56222617
11 [0.46675394 0.54312794 0.77152047] 0.60895537
12 [0.48062598 0.53573737 0.77483515] 0.60797909
13 [0.4451194  0.58078192 0.77118154] 0.59991625
14 [0.43744346 0.54


After truncation: ((358, 538), (358,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[219]	training's auc: 0.899228	valid_1's auc: 0.826421
logloss = 	 0.5171439917211885
ROC = 	 0.8264214919144497
0 [0.51207097 0.5741945  0.75119186] 0.55706988
1 [0.38254212 0.68543595 0.75169056] 0.57489493
2 [0.51409389 0.51843684 0.68864197] 0.53926847
3 [0.50907681 0.51580197 0.72345604] 0.54943987
4 [0.49411722 0.56691356 0.72292087] 0.55645622
5 [0.32473167 0.60260038 0.72281317] 0.55650522
6 [0.51385481 0.55429579 0.72231764] 0.55323682
7 [0.38411358 0.58852801 0.75183236] 0.56169346
8 [0.54707401 0.65549562 0.75127064] 0.57252882
9 [0.55099747 0.65542738 0.7224825 ] 0.56961123
10 [0.31949023 0.63388655 0.76354321] 0.56299933
11 [0.31353632 0.63853648 0.7512626 ] 0.57108549
12 [0.51978181 0.65047666 0.72354672] 0.57140414
13 [0.37828967 0.67877132 0.75147377] 0.5774584
14 [0.37746021 0.68035627 0.72279202] 0.57554724
15 [0.51217146 0.64334732 


After truncation: ((358, 538), (358,))
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[219]	training's auc: 0.899228	valid_1's auc: 0.816474
logloss = 	 0.5251441239868503
ROC = 	 0.8164738950633239
0 [0.40713327 0.57553141 0.75224594] 0.5561367
1 [0.46028618 0.5619031  0.69466129] 0.53748067
2 [0.38959888 0.56439068 0.75295699] 0.55349875
3 [0.396477   0.61439249 0.69372431] 0.55186073
4 [0.3628333  0.64675642 0.69404877] 0.55156212
5 [0.52436283 0.57442157 0.73718046] 0.54564918
6 [0.3852298  0.51545126 0.68675088] 0.54124549
7 [0.52001658 0.5747181  0.68998024] 0.54648592
8 [0.38268249 0.56180126 0.69409607] 0.553471
9 [0.45886179 0.51971815 0.69194636] 0.52853675
10 [0.38940215 0.6092643  0.69411241] 0.55405905
11 [0.45821062 0.56172481 0.69466827] 0.53748067
12 [0.49280287 0.56049629 0.69507307] 0.54055609
13 [0.51585999 0.52979781 0.69475392] 0.54078267
14 [0.4937906  0.56668551 0.68696975] 0.53741711
15 [0.38324792 0.56290389 0.

3    0.466
2    0.221
0    0.183
1    0.130
Name: accuracy_group, dtype: float64

In [18]:
# https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
from collections import defaultdict

def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices
        
def modelling_sgk_reg(train, test, feat, select_flg): # stratify by target, group by session_id
    lgbm_params = {'objective': 'regression', 'boosting_type': 'gbdt',
     'tree_learner': 'serial','bagging_fraction': 0.5698056418890787,'bagging_freq': 4,
     'colsample_bytree': 0.37564408454469,'learning_rate': 0.015433389422506185,'max_depth': 8,
     'min_data_in_leaf': 51,'min_sum_hessian_in_leaf': 10,'num_leaves': 48}

    y_train = train.accuracy_group.copy()
    X_train = train.drop(["accuracy_group"],axis=1) 
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    groups = np.array(X_train.installation_id.values)

    models = []
    random_try = 30
    n_folds = 10
    evals_result = {}
    mean_qwk_score = 0
    for i, (train_index, test_index) in enumerate(stratified_group_k_fold(X_train, y_train, groups, k=n_folds, seed=12)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2.drop("installation_id", axis=1, inplace=True)
        
        for try_time in range(random_try):
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            # random truncation ------------------
            X_test2, idx_val = get_random_assessment(X_test2)
            X_test2.drop('installation_id', inplace=True, axis=1)
            y_test2 = y_test2.loc[idx_val]
            # ------------------------------------

            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500, categorical_feature = categoricals)
            valid = np.array(clf.predict(X_test2, num_iteration = clf.best_iteration))
            real = np.array(y_test2)

            models.append(clf)
            rmse = np.sqrt(mean_squared_error(real, valid))
            print("RMSE = {}".format(rmse))
            
            # threshold optimization --------------
            best_score = 0
            for j in range(20):
                optR = OptimizedRounder_reg()
                optR.fit(np.array(valid).reshape(-1,), real, random_flg=True)
                coefficients = optR.coefficients()
                final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
                score = qwk(real, final_valid_pred)
                print(j, np.sort(coefficients), score)
                if score > best_score:
                    best_score = score
                    best_coefficients = coefficients
            mean_qwk_score += best_score / (random_try * n_folds)
            if try_time == 0 and i == 0:
                final_coefficients = np.sort(best_coefficients) / (random_try * n_folds)
            else:
                final_coefficients += np.sort(best_coefficients) / (random_try * n_folds)

    print("mean QWK = {}".format(mean_qwk_score))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return mean_qwk_score, pred_value, final_coefficients
    
#qwk_sgk1, pred_value, final_coefficients = modelling_sgk_reg(mod_train, new_test, feat, False)
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(final_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.32229148, 0.51887455, 0.77529457])) + [np.inf], labels = [0, 1, 2, 3])

#sample_submission["accuracy_group"] = final_test_pred.astype(int)
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission["accuracy_group"].value_counts(normalize = True)

In [19]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': "auc", 
        'objective': 'binary', 
        'eval_metric': 'cappa', 
        "tree_learner": "serial",
        'max_depth': int(params['max_depth']),
        'bagging_freq': int(params['bagging_freq']),
        'bagging_fraction': float(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'learning_rate': float(params['learning_rate']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
}
    
        real = np.array([])
        pred = np.array([])
        skf = GroupKFold(n_splits=10)
        for trn_idx, val_idx in skf.split(X, Y, X["installation_id"]):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            x_val['accuracy_group'] = y_val
            np.random.seed(0)
            x_val_mod = x_val.groupby('installation_id').agg(np.random.choice).reset_index(drop=False)
            y_val_mod = x_val_mod.accuracy_group.copy()
            x_train.drop('installation_id', inplace = True, axis = 1)
            x_val_mod.drop(['installation_id', "accuracy_group"], inplace = True, axis = 1)
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = ['session_title'])
            val_set = lgb.Dataset(x_val_mod, y_val_mod, categorical_feature = ['session_title'])
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                         valid_sets = [train_set, val_set], verbose_eval = 300)
            pred = np.concatenate((pred, np.array(clf.predict(x_val_mod, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val_mod)), axis=0) 
        score = roc_auc_score(real, pred)
    
        return - score

    trials = Trials()

    space ={
        'max_depth': hp.quniform('max_depth', 1, 15, 1),
        'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.2, 1.0),
        'num_leaves': hp.quniform('num_leaves', 8, 64, 1),
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 8, 64, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 30, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=10, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#X_train = new_train.drop(["accuracy_group"], axis=1).copy()
#Y = new_train.accuracy_group.copy()
#Y.loc[Y <=1] = 0
#Y.loc[Y >=2] = 1
#lbl = preprocessing.LabelEncoder()
#lbl.fit(list(X_train["installation_id"]))
#X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
#remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
#                      or i in to_exclude]
#for i in X_train.columns:
#    if X_train[i].std() == 0 and i not in remove_features:
#        remove_features.append(i)
#X_train = X_train.drop(remove_features, axis=1)
#X_train = X_train[sorted(X_train.columns.tolist())]

#random_state = 42
#my_hyperopt(X_train, Y)